In [6]:
import numpy as np
from scapy.all import sniff
from keras.models import load_model

# Önceden eğitilmiş modeli yükleme
model = load_model('lstm_model.h5')  # Model dosyasının adı

# Protokol numarasını insan okuyabilir bir dizeye dönüştürme
def get_protocol_name(proto):
    if proto == 6:
        return 'TCP'
    elif proto == 17:
        return 'UDP'
    else:
        return 'Other'

# Paketlerden özellikleri çıkarma ve modelle tahmin yapma işlemi için callback fonksiyonu
def packet_callback(packet):
    try:
        if packet.haslayer("IP"):
            # Paket özelliklerini çıkarma
            protocol_type = packet["IP"].proto  # Protokol numarası
            src_ip = packet["IP"].src
            dst_ip = packet["IP"].dst
            packet_length = len(packet)  # Paket boyutu

            # Protokol türünü kodlama
            protocol_type_encoded = {
                6: 1,   # TCP
                17: 2,  # UDP
            }.get(protocol_type, 0)  # Diğer protokoller için varsayılan değer 0

            # Özellik vektörünü oluşturma
            feature_vector = np.array([packet_length, protocol_type_encoded]).reshape(1, -1)

            # LSTM modelinin girişi için yeniden şekillendirme
            feature_vector = np.reshape(feature_vector, (1, feature_vector.shape[1], 1))

            # Model ile tahmin yapma
            prediction = model.predict(feature_vector, verbose=0)

            # Tahmin sonucunu değerlendirme
            protocol_name = get_protocol_name(protocol_type)
            if prediction[0][0] > 0.5:  # Tahmin eşiği (0.5 olarak ayarlandı)
                print(f"Tahmin: Anormal (Attack) | Source: {src_ip}, Destination: {dst_ip}, Protocol: {protocol_name}, Paket Boyutu: {packet_length}")
            else:
                print(f"Tahmin: Normal (No Attack) | Source: {src_ip}, Destination: {dst_ip}, Protocol: {protocol_name}, Paket Boyutu: {packet_length}")

        else:
            print("Paketin IP katmanı yok, atlanıyor.")

    except Exception as e:
        print(f"Paket işlenirken hata oluştu: {e}")

# Gerçek zamanlı trafik yakalamak için sniff fonksiyonu
try:
    sniff(iface='Wi-Fi', prn=packet_callback, store=0)  # 'Wi-Fi' yerine ağ arayüzünüzün adını yazın
except Exception as e:
    print(f"Sniffing sırasında hata oluştu: {e}")


Tahmin: Normal (No Attack) | Source: 13.78.111.199, Destination: 192.168.2.248, Protocol: TCP, Paket Boyutu: 153
Tahmin: Normal (No Attack) | Source: 192.168.2.248, Destination: 13.78.111.199, Protocol: TCP, Paket Boyutu: 491
Tahmin: Normal (No Attack) | Source: 13.78.111.199, Destination: 192.168.2.248, Protocol: TCP, Paket Boyutu: 1454
Tahmin: Normal (No Attack) | Source: 13.78.111.199, Destination: 192.168.2.248, Protocol: TCP, Paket Boyutu: 1454
Tahmin: Normal (No Attack) | Source: 13.78.111.199, Destination: 192.168.2.248, Protocol: TCP, Paket Boyutu: 1454
Tahmin: Normal (No Attack) | Source: 192.168.2.248, Destination: 13.78.111.199, Protocol: TCP, Paket Boyutu: 54
Tahmin: Normal (No Attack) | Source: 13.78.111.199, Destination: 192.168.2.248, Protocol: TCP, Paket Boyutu: 344
Tahmin: Normal (No Attack) | Source: 192.168.2.248, Destination: 13.78.111.199, Protocol: TCP, Paket Boyutu: 54
Tahmin: Normal (No Attack) | Source: 192.168.2.248, Destination: 13.78.111.199, Protocol: TCP, 

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as ts
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, MaxPool1D, Flatten,Dropout,Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D
import numpy as np
df_anomaly = pd.read_csv('data.csv')
df_anomaly.head()
df_anomaly.columns = df_anomaly.columns.str.strip().str.replace('"', '')
traffic_summary = df_anomaly.groupby(["Source", "Destination"]).size().reset_index(name="Packet Count")
attack_threshold = 1000
traffic_summary['Label'] = (traffic_summary['Packet Count'] > attack_threshold).astype(int)
def label_row(row):
    attack_protocols = ["UDP", "TCP"]  # Saldırılarda yaygın kullanılan protokoller
    if row["Protocol"] in attack_protocols and row["Length"] > 1000:
        return 1  # Saldırı
    else:
        return 0  # Normal
data = df_anomaly.merge(
    traffic_summary[['Source', 'Destination', 'Label']],
    on=['Source', 'Destination'],
    how='left'
)
data["Label"] = data.apply(label_row, axis=1)
output_path = 'anomaly.csv'
data.to_csv(output_path, index=False)
df_normal = pd.read_csv('ağ.csv')
df_normal.head()
df_normal.columns = df_normal.columns.str.strip().str.replace('"', '')
traffic_summary = df_normal.groupby(["Source", "Destination"]).size().reset_index(name="Packet Count")
attack_threshold = 1000
traffic_summary['Label'] = (traffic_summary['Packet Count'] > attack_threshold).astype(int)
def label_row(row):
    attack_protocols = ["UDP", "TCP"]  # Saldırılarda yaygın kullanılan protokoller
    if row["Protocol"] in attack_protocols and row["Length"] > 1000:
        return 1  # Saldırı
    else:
        return 0  # Normal
data_normal = df_normal.merge(
    traffic_summary[['Source', 'Destination', 'Label']],
    on=['Source', 'Destination'],
    how='left'
)
data_normal["Label"] = data_normal.apply(label_row, axis=1)
output_path = 'normal.csv'
data.to_csv(output_path, index=False)
merged_df = pd.concat([data, data_normal], axis=0)
merged_df.to_csv('dataset.csv', index=False)
X = merged_df.drop(columns=["Label"])  # Özellikler: Etiket sütunu hariç tüm sütunlar
y = merged_df["Label"]  # Etiketler: Saldırı (1) veya Normal (0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Eğitim seti boyutu: {len(X_train)}")
print(f"Test seti boyutu: {len(X_test)}")
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)
print(X_train.dtypes)
print(y_train.dtypes)
import pandas as pd  
from keras.models import Sequential  
from keras.layers import Dense  

# 1. CSV dosyasını okuma  
X_test = pd.read_csv('X_test.csv')  

# 2. Time sütununu datetime formatına dönüştürme  
X_test['Time'] = pd.to_datetime(X_test['Time'], format='%Y-%m-%d %H:%M:%S,%f')  

# 3. Tarih özelliklerini çıkarma  
X_test['year'] = X_test['Time'].dt.year  
X_test['month'] = X_test['Time'].dt.month  
X_test['day'] = X_test['Time'].dt.day  
X_test['hour'] = X_test['Time'].dt.hour  
X_test['minute'] = X_test['Time'].dt.minute  
X_test['second'] = X_test['Time'].dt.second  
X_test['microsecond'] = X_test['Time'].dt.microsecond  

# 4. Sayısal olmayan sütunları çıkartma  
non_numeric_cols = ['No.', 'Time', 'Source', 'Destination', 'Protocol', 'Info']  
X_test = X_test.drop(columns=non_numeric_cols)  

# 5. Diğer boolean türündeki sütunları dönüştürme (eğer varsa)  
if 'some_boolean_column' in X_test.columns and X_test['some_boolean_column'].dtype == 'bool':  
    X_test['some_boolean_column'] = X_test['some_boolean_column'].astype(int)  

# 6. Sayısal sütunları float türüne dönüştürme  
numeric_cols = X_test.select_dtypes(include=['float64', 'int64']).columns  
X_test[numeric_cols] = X_test[numeric_cols].apply(pd.to_numeric, errors='coerce')  
import pandas as pd  
from keras.models import Sequential  
from keras.layers import Dense  

# 1. CSV dosyasını okuma  
X_train = pd.read_csv('X_train.csv')  

# 2. Time sütununu datetime formatına dönüştürme  
X_train['Time'] = pd.to_datetime(X_train['Time'], format='%Y-%m-%d %H:%M:%S,%f')  

# 3. Tarih özelliklerini çıkarma  
X_train['year'] = X_train['Time'].dt.year  
X_train['month'] = X_train['Time'].dt.month  
X_train['day'] = X_train['Time'].dt.day  
X_train['hour'] = X_train['Time'].dt.hour  
X_train['minute'] = X_train['Time'].dt.minute  
X_train['second'] = X_train['Time'].dt.second  
X_train['microsecond'] = X_train['Time'].dt.microsecond  

# 4. Sayısal olmayan sütunları çıkartma  
non_numeric_cols = ['No.', 'Time', 'Source', 'Destination', 'Protocol', 'Info']  
X_train = X_train.drop(columns=non_numeric_cols)  

# 5. Diğer boolean türündeki sütunları dönüştürme (eğer varsa)  
if 'some_boolean_column' in X_train.columns and X_train['some_boolean_column'].dtype == 'bool':  
    X_train['some_boolean_column'] = X_train['some_boolean_column'].astype(int)  

# 6. Sayısal sütunları float türüne dönüştürme  
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns  
X_train[numeric_cols] = X_train[numeric_cols].apply(pd.to_numeric, errors='coerce')  
print(X_train.dtypes)
print(X_test.dtypes)
model = Sequential() 
model.add(Conv1D(32, 9, padding="same",input_shape = (X_train.shape[1], 1), activation='relu'))
model.add(MaxPool1D(pool_size=(2)))
model.add(LSTM(units=16,return_sequences=False,dropout=0.2))
model.add(Dense(units=1))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=250)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
Y_test = np.array(y_test)

test_results = model.evaluate(X_test, Y_test, verbose=1)
import numpy as np  
import random  
import pandas as pd  

# Test veri setinden rastgele bir satır seç  
random_index = random.randint(0, X_test.shape[0] - 1)  
random_sample = X_test[random_index]  

# Model giriş boyutlarına uygun hale getirme  
random_sample_reshaped = np.reshape(random_sample, (1, random_sample.shape[0], 1))  

# Model tahmini yap  
prediction = model.predict(random_sample_reshaped, verbose=0)  

# Tahmini normal veya anormal olarak etiketleme  
if prediction[0][0] > 0.5:  
    print("Tahmin: Anormal (Attack)")  
else:  
    print("Tahmin: Normal (No Attack)")  

# Gerçek etiketi göster  
true_label = y_test.iloc[random_index]  # y_test bir pandas Series olduğu için iloc ile erişiyoruz.  

if true_label == 1:  
    true_label_text = 'Anormal (Attack)'  
else:  
    true_label_text = 'Normal (No Attack)'  

print(f"Gerçek Etiket: {true_label_text}")
model.save('lstm_model.h5')
from scapy.all import sniff  
import numpy as np  
from keras.models import load_model  

# Önceden eğitilmiş modeli yükleyin  
model = load_model('lstm_model.h5')  # Model dosyanızın adını doğru belirtin  

# Protokol numarasını insan okuyabilir bir dizeye dönüştürme  
def get_protocol_name(proto):  
    if proto == 6:  
        return 'TCP'  
    elif proto == 17:  
        return 'UDP'  
    else:  
        return 'Diğer'  

# Paketi yazdırmak için bir callback fonksiyonu  
def packet_callback(packet):  
    try:  
        # Protokol ve kaynak/destination bilgilerini çıkarma  
        if packet.haslayer("IP"):  
            protocol_type = packet[0][1].proto  # Protokol numarasını al  
            src_ip = packet[0][1].src  
            dst_ip = packet[0][1].dst  
            src_bytes = len(packet)  # Paket boyutu  
            dst_bytes = 0  # Bu değeri ihtiyaçlarınıza göre ayarlayabilirsiniz  
        else:  
            src_ip = "No IP Layer"  
            dst_ip = "No IP Layer"  
            src_bytes = 0  
            dst_bytes = 0  
            protocol_type = 0  # Protokol bilgisi yok  

        # Özellik vektörünü oluşturma  
        # Protokol tiplerine göre kodlama  
        protocol_type_encoded = {  
            6: 1,   # TCP  
            17: 2,  # UDP  
        }.get(protocol_type, 0)  # Diğer tüm protokoller için varsayılan 0  

        feature_vector = np.array([src_bytes, dst_bytes, protocol_type_encoded]).reshape(1, -1)  

        # LSTM modelinin girişi için yeniden şekillendirme  
        feature_vector = np.reshape(feature_vector, (1, feature_vector.shape[1], 1))  

        # Model tahmini  
        prediction = model.predict(feature_vector, verbose=0)  
        
        # Protokol ismini al  
        protocol_name = get_protocol_name(protocol_type)  

        # Tahmini eşiğine göre sonuç  
        if prediction[0][0] > 0.5:  # Eşik değeri  
            print(f"Tahmin: Anormal (Attack) | Source: {src_ip}, Destination: {dst_ip}, Protocol: {protocol_name}, Paket Boyutu: {src_bytes}")  
        else:  
            print(f"Tahmin: Normal (No Attack) | Source: {src_ip}, Destination: {dst_ip}, Protocol: {protocol_name}, Paket Boyutu: {src_bytes}")  

    except Exception as e:  
        print(f"Bu paketten özellikler çıkarılamadı. Hata: {e}")  

# Paket yakalamak için sniff fonksiyonunu kullanıyoruz  
sniff(iface='Wi-Fi', prn=packet_callback)

Eğitim seti boyutu: 8734
Test seti boyutu: 2184
No.             int64
Time           object
Source         object
Destination    object
Protocol       object
Length          int64
Info           object
dtype: object
int64
Length         int64
year           int32
month          int32
day            int32
hour           int32
minute         int32
second         int32
microsecond    int32
dtype: object
Length         int64
year           int32
month          int32
day            int32
hour           int32
minute         int32
second         int32
microsecond    int32
dtype: object


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2:32 4s/step - accuracy: 0.3520 - loss: 8.460 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3860 - loss: 8.488 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3869 - loss: 8.479 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3881 - loss: 8.446 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3894 - loss: 8.415 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3917 - loss: 8.354 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3936 - loss: 8.295 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3950 - loss: 8.243 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3967 - loss: 8.185 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.3971 - loss: 8.1722
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.4320 - loss: 7.344 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4442 - loss: 7.026 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4399 - loss: 6.930 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4411 - loss: 6.819 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms

<Sniffed: TCP:31 UDP:16 ICMP:0 Other:0>

In [8]:
from scapy.all import sniff
import numpy as np
from keras.models import load_model
import time

# Önceden eğitilmiş modeli yükleyin
model = load_model('lstm_model.h5')  # Model dosyanızın adı doğru olmalı

# DoS saldırısı için bir eşik değer tanımlayın
DOS_ATTACK_THRESHOLD = 0.5  # Modelin tahmin eşiği
ATTACK_COUNT_LIMIT = 10  # Belirli bir süre içinde kaç saldırının alarm oluşturacağını belirler
MONITOR_WINDOW = 10  # Saniye cinsinden izleme penceresi

# Saldırı sayacı ve zamanlayıcı
attack_counter = 0
start_time = time.time()

# Protokol numarasını insan okuyabilir bir dizeye dönüştürme
def get_protocol_name(proto):
    if proto == 6:
        return 'TCP'
    elif proto == 17:
        return 'UDP'
    else:
        return 'Diğer'

# Paketi analiz etmek için bir callback fonksiyonu
def packet_callback(packet):
    global attack_counter, start_time

    try:
        # Protokol ve kaynak/destination bilgilerini çıkarma
        if packet.haslayer("IP"):
            protocol_type = packet[0][1].proto  # Protokol numarasını al
            src_ip = packet[0][1].src
            dst_ip = packet[0][1].dst
            src_bytes = len(packet)  # Paket boyutu
            dst_bytes = 0  # Bu değeri ihtiyaçlarınıza göre ayarlayabilirsiniz
        else:
            src_ip = "No IP Layer"
            dst_ip = "No IP Layer"
            src_bytes = 0
            dst_bytes = 0
            protocol_type = 0  # Protokol bilgisi yok

        # Özellik vektörünü oluşturma
        protocol_type_encoded = {
            6: 1,   # TCP
            17: 2,  # UDP
        }.get(protocol_type, 0)  # Diğer tüm protokoller için varsayılan 0

        feature_vector = np.array([src_bytes, dst_bytes, protocol_type_encoded]).reshape(1, -1)

        # LSTM modelinin girişi için yeniden şekillendirme
        feature_vector = np.reshape(feature_vector, (1, feature_vector.shape[1], 1))

        # Model tahmini
        prediction = model.predict(feature_vector, verbose=0)

        # Protokol ismini al
        protocol_name = get_protocol_name(protocol_type)

        # Tahmini eşiğine göre sonuç
        if prediction[0][0] > DOS_ATTACK_THRESHOLD:  # Anormal paket tespiti
            print(f"ALARM: Anormal Trafik! | Source: {src_ip}, Destination: {dst_ip}, Protocol: {protocol_name}, Paket Boyutu: {src_bytes}")
            attack_counter += 1
        else:
            print(f"Normal Trafik | Source: {src_ip}, Destination: {dst_ip}, Protocol: {protocol_name}, Paket Boyutu: {src_bytes}")

        # İzleme penceresindeki saldırı sayısını kontrol et
        current_time = time.time()
        if current_time - start_time > MONITOR_WINDOW:  # İzleme penceresi süresi dolduğunda
            if attack_counter >= ATTACK_COUNT_LIMIT:
                print(f"Uyarı: {MONITOR_WINDOW} saniyede {attack_counter} saldırı tespit edildi! DoS saldırısı olabilir!")
            # Sayaçları sıfırla
            attack_counter = 0
            start_time = current_time

    except Exception as e:
        print(f"Bu paketten özellikler çıkarılamadı. Hata: {e}")

# Paket yakalamak için sniff fonksiyonunu kullanıyoruz
sniff(iface='Wi-Fi', prn=packet_callback, store=False)


Normal Trafik | Source: 13.107.246.44, Destination: 192.168.2.248, Protocol: TCP, Paket Boyutu: 93
Normal Trafik | Source: 192.168.2.248, Destination: 13.107.246.44, Protocol: TCP, Paket Boyutu: 54
Normal Trafik | Source: 13.107.246.44, Destination: 192.168.2.248, Protocol: TCP, Paket Boyutu: 78
Normal Trafik | Source: 192.168.2.248, Destination: 13.107.246.44, Protocol: TCP, Paket Boyutu: 54
Normal Trafik | Source: 13.107.246.44, Destination: 192.168.2.248, Protocol: TCP, Paket Boyutu: 54
Normal Trafik | Source: 192.168.2.248, Destination: 20.189.173.15, Protocol: TCP, Paket Boyutu: 172
Normal Trafik | Source: 192.168.2.248, Destination: 20.189.173.15, Protocol: TCP, Paket Boyutu: 1454
Normal Trafik | Source: 192.168.2.248, Destination: 20.189.173.15, Protocol: TCP, Paket Boyutu: 1454
Normal Trafik | Source: 192.168.2.248, Destination: 20.189.173.15, Protocol: TCP, Paket Boyutu: 1454
Normal Trafik | Source: 192.168.2.248, Destination: 20.189.173.15, Protocol: TCP, Paket Boyutu: 1454
N

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>